# Library import

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Any results you write to the current directory are saved as output.

import matplotlib.pyplot as plt

In [0]:
tf.__version__

# Data

In [0]:
path = '../input/digit-recognizer/'

train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [0]:
Counter(train['label']) # label 0 ~ 9 : 10개

# Reshape

In [0]:
x_train = np.reshape(train.drop('label', axis=1).values, [-1, 28, 28])
y_train = train.label.values
x_test = np.reshape(test.values, [-1, 28, 28])
x_train.shape, y_train.shape, x_test.shape # train 42000, test 28000 data

# Expand dimension

In [0]:
x_train_set = np.expand_dims(x_train, axis=3)
x_test_set = np.expand_dims(x_test, axis=3)
x_train_set.shape, x_test_set.shape

# normalization

In [0]:
x_train_set = x_train_set/255.
x_test_set = x_test_set/255.

# Train, test split

In [0]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x_train_set, y_train)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

# Image Augmentation

In [0]:
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_datagen.fit(train_x)

# Model

In [0]:
model = models.Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1), kernel_initializer='he_normal'))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.3))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = "softmax"))
model.summary()
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# Training

In [0]:
epochs = 8
batch_size = 256

history = model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=batch_size),
                              epochs = epochs, validation_data = (test_x, test_y),
                              verbose = 2, steps_per_epoch=train_x.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

# Submission

In [0]:
predict = np.argmax(model.predict(x_test_set), axis=-1)
submission['Label'] = predict
submission.to_csv('sample_submission.csv', index=False)